
# **Задача 4**

## Вариант 1

### Условие
* Распределение: Показательное (экспоненциальное) — $Exp(\lambda)$.
* Параметр: Медиана ($m$).
* Значение параметра для эксперимента: $\lambda = 1$.
* Указание: Воспользуйтесь предельной теоремой об асимптотическом поведении среднего члена вариационного ряда.

Функция распределения для $Exp(\lambda)$:
$$ F(x) = 1 - e^{-\lambda x} \quad (\text{при } x \ge 0) $$

Медиану $m$ определяем из условия $F(m) = 0.5$:
$$ 1 - e^{-\lambda m} = 0.5 \implies e^{-\lambda m} = 0.5 \implies -\lambda m = \ln(0.5) \implies m = \frac{\ln 2}{\lambda} $$

$\lambda$ через $m$:
$$ \lambda = \frac{\ln 2}{m} $$

Пусть $\hat{m}$ - выборочная медиана (средний член вариационного ряда). Согласно теореме об асимптотическом поведении выборочных квантилей:
$$ \sqrt{n}(\hat{m} - m) \xrightarrow{d} N\left(0, \frac{1}{4 [f(m)]^2}\right) $$

$f(x) = \lambda e^{-\lambda x}$ (плотность распределения).
Найдем значение плотности в точке медианы:
$$ f(m) = \lambda e^{-\lambda \cdot \frac{\ln 2}{\lambda}} = \lambda e^{-\ln 2} = \lambda \cdot \frac{1}{2} = \frac{\lambda}{2} $$

Подставим это в формулу асимптотической дисперсии:
$$ \sigma_{asymp}^2 = \frac{1}{4 \cdot (\frac{\lambda}{2})^2} = \frac{1}{4 \cdot \frac{\lambda^2}{4}} = \frac{1}{\lambda^2} $$

Таким образом, распределение выборочной медианы при больших $n$ приближается к нормальному:
$$ \hat{m} \sim N\left(m, \frac{1}{n \lambda^2}\right) $$

Нам нужно построить ДИ для $m$. В формулу дисперсии входит неизвестный параметр $\lambda$.

Мы можем выразить $\lambda$ через саму медиану, воспользовавшись свойством состоятельности $\hat{m} \xrightarrow{P} m$:
$$ \frac{1}{\lambda} = \frac{m}{\ln 2} $$

Тогда теоретическая стандартная ошибка (Standard Error) равна:
$$ SE(\hat{m}) = \frac{1}{\lambda \sqrt{n}} = \frac{m}{\ln 2 \sqrt{n}} $$

Заменяем неизвестное теоретическое $m$ на его оценку $\hat{m}$ (выборочную медиану) для получения оценки стандартной ошибки:
$$ \widehat{SE} = \frac{\hat{m}}{\ln 2 \sqrt{n}} $$

**Итоговый асимптотический ДИ уровня $1-\alpha$:**
$$ \left( \hat{m} - z_{1-\alpha/2} \frac{\hat{m}}{\ln 2 \sqrt{n}}; \quad \hat{m} + z_{1-\alpha/2} \frac{\hat{m}}{\ln 2 \sqrt{n}} \right) $$

где $z_{1-\alpha/2}$ - квантиль стандартного нормального распределения

In [1]:
import numpy as np
from scipy.stats import norm

In [3]:
true_lambda = 1
true_median = np.log(2) / true_lambda

alpha = 0.05
z_crit = norm.ppf(1 - alpha / 2)
N_iter = 1000

def experiment(n):
    cover_count = 0
    lengths = []

    for _ in range(N_iter):
        sample = np.random.exponential(scale=1/true_lambda, size=n)

        m_hat = np.median(sample)

        se = m_hat / (np.log(2) * np.sqrt(n))

        ci_low = m_hat - z_crit * se
        ci_high = m_hat + z_crit * se

        lengths.append(ci_high - ci_low)

        if ci_low <= true_median <= ci_high:
            cover_count += 1

    return cover_count / N_iter, np.mean(lengths)

In [4]:
cov_25, length_25 = experiment(25)
cov_10000, length_10000 = experiment(10000)

In [6]:
print(f"Истинная медиана: {true_median:.5f}")
print("=== РЕЗУЛЬТАТЫ ===")
print(f"n = 25:")
print(f"  Покрытие (target 0.95): {cov_25:.3f}")
print(f"  Средняя длина интервала: {length_25:.3f}\n")

print(f"n = 10000:")
print(f"  Покрытие (target 0.95): {cov_10000:.3f}")
print(f"  Средняя длина интервала: {length_10000:.6f}")

Истинная медиана: 0.69315
=== РЕЗУЛЬТАТЫ ===
n = 25:
  Покрытие (target 0.95): 0.925
  Средняя длина интервала: 0.809

n = 10000:
  Покрытие (target 0.95): 0.929
  Средняя длина интервала: 0.039179
